In [ ]:
from pydantic import BaseModel, Field, HttpUrl
from typing import List
import configparser

# Define Pydantic models for each config section
class YouTrackConfig(BaseModel):
    Token: str
    BaseUrl: str

class VsemRabotaConfig(BaseModel):
    ProjectId: str
    WBTeam: List[str] = Field(default_factory=list)
    Name: str

# Main configuration model that includes both sections
class AppConfig(BaseModel):
    YouTrack: YouTrackConfig
    VsemRabota: VsemRabotaConfig

# Parse the INI file
def parse_config_to_model(config_file: str) -> AppConfig:
    config = configparser.ConfigParser()
    config.read(config_file)
    
    # YouTrack section
    youtrack_token = config.get('YouTrack', 'Token').strip("'")
    youtrack_base_url = config.get('YouTrack', 'BaseUrl')
    
    # VsemRabota Config section
    project_id = config.get('VsemRabota Config', 'ProjectId')
    name = config.get('VsemRabota Config', 'Name')
    wb_team = config.get('VsemRabota Config', 'WBTeam').split(', ')
    
    # Create instances of the models
    youtrack_config = YouTrackConfig(Token=youtrack_token, BaseUrl=youtrack_base_url)
    vsem_rabota_config = VsemRabotaConfig(ProjectId=project_id, WBTeam=wb_team, Name=name)
    
    # Aggregate into the main config model
    app_config = AppConfig(YouTrack=youtrack_config, VsemRabota=vsem_rabota_config)
    
    return app_config

In [ ]:
config_file = 'yt.ini'  # Update this path
config_model = parse_config_to_model(config_file)
print(config_model)

In [ ]:
import requests

headers = {
    'Authorization': f'Bearer {config_model.YouTrack.Token}',
    'Content-Type': 'application/json',
}

search_query = 'Business Line:HR   State: Resolved sort by: {Threat Score} desc'

response = requests.get(f'{config_model.YouTrack.Token}/api/issues?query={search_query}', headers=headers)

### Client

In [30]:
import requests
from urllib.parse import urlencode

class YouTrackAPIError(Exception):
    """Exception raised for errors in the YouTrack API."""
    def __init__(self, status_code, message):
        super().__init__(f"HTTP {status_code}: {message}")
        self.status_code = status_code
        self.message = message

class YouTrackClient:
    def __init__(self, config):
        self.config = config
        self.headers = {
            'Authorization': f'Bearer {self.config.Token}',
            'Content-Type': 'application/json',
        }

    #     headers = {
    #     'Authorization': f'Bearer {api_token}',
    #     'Content-Type': 'application/json',
    # }

    def get(self, endpoint: str, params: dict = None) -> dict:
        """Sends a GET request to a specified endpoint with optional query parameters."""
        url = f"{self.config.BaseUrl}{endpoint}"
        # param_str = '&'.join([f'{key}={value}' for key, value in params.items()])
        if params:
            url += f"?{urlencode(params)}"
        try:
            response = requests.get(url, headers=self.headers)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.HTTPError as e:
            raise YouTrackAPIError(response.status_code, response.text) from e

    def create_issue_with_custom_fields(self, issue_custom_fields, project_id, summary, description):
        serialized_fields = issue_custom_fields.serialize()
        
        data = {
            "project": {"id": project_id},
            "summary": summary,
            "description": description,
            "customFields": serialized_fields,
        }

        return self.post('/api/issues', data=data)

    def post(self, endpoint: str, data: dict = None) -> dict:
        """Sends a POST request to a specified endpoint with optional JSON data."""
        url = f"{self.config.BaseUrl}{endpoint}"
        try:
            response = requests.post(url, json=data, headers=self.headers)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.HTTPError as e:
            raise YouTrackAPIError(response.status_code, response.text) from e

    def get_issues(self, search_query: str, additional_params: dict = None) -> dict:
        """
        Retrieves issues from YouTrack based on a search query and additional parameters.

        :param search_query: A string representing the search query for filtering issues.
        :param additional_params: Optional dictionary of additional query parameters.
        :return: A dictionary containing the API response with issues.
        """
        params = {'query': search_query}
        if additional_params:
            params.update(additional_params)
        
        return self.get('/api/issues', params=params)

    def get_issue(self, issue_id, fields=None):
        issue_fields = [
            'id', 'idReadable', 'summary', 'description', 'reporter(id,login,fullName)',
            'customFields($type,id,projectCustomField($type,id,field($type,id,name)),value($type,avatarUrl,buildLink,color(id),fullName,id,isResolved,localizedName,login,minutes,name,presentation,text))',
            'linkType(name,sourceToTarget,targetToSource)',
            'issues(id,idReadable,summary)'
        ]
        
        if fields is None:
            fields = 'id,idReadable,summary,description,customFields(id,projectCustomField(field(name))),linkType(name,sourceToTarget,targetToSource),issues(id,idReadable,summary)'
            fields = ','.join(issue_fields)

        params = {
            'fields': fields
        }

        return self.get(f'/api/issues/{issue_id}', params=params)

    def get_field(self, issue_id, field_id):
        params = {'fields': 'id,projectCustomField(id,field(id,name)),value(id,isResolved,localizedName,name)'}
        return self.get(f'/api/issues/{issue_id}/customFields/{field_id}', params=params)

    def get_issue_history(self, issue_id):
        'api/issues/SP-32/activities'

        params = {
            'categories': 'CustomFieldCategory,CommentsCategory',
            'fields': 'author(name,login),timestamp,target(text),added(name),removed(name)'
        }
        return self.get(f'/api/issues/{issue_id}/activities', params=params)

    def get_projects(self, additional_params: dict = None):
        leader_fields = ['login', 'name', 'id']
        createdby_fields = ['login', 'name', 'id']
        
        params = {'fields': 'id,name,shortName,createdBy(login,name,id),leader(login,name,id),key'}
        if additional_params:
            params.update(additional_params)
        
        return self.get('/api/admin/projects', params=params)

    def get_project_by_name(self, shortName: str):
        params = {
            'fields': 'id,name,shortName',
            'query': shortName
        }

        return self.get('/api/admin/projects', params=params)
    

### Initialize

In [27]:
youtrack_client = YouTrackClient(config=config_model.YouTrack)
        

In [ ]:
resp = youtrack_client.get_issue('VR-2934')

# resp['customFields']
resp


In [ ]:
def get_customfield_value(customfield, issue):
    return next(r['value'] for r in issue['customFields'] if r['projectCustomField']['field']['name'] == customfield)

def get_issue_authority(issue):
    assignee = get_customfield_value('Assignee', issue)
    reviewer = get_customfield_value('Reviewer [HR]', issue)
    product = get_customfield_value('Product', issue)
    reporter = resp['reporter']['fullName']

    return {
        'assignee': assignee['fullName'] if assignee else None,
        'reviewer': reviewer['fullName'] if reviewer else None,
        'products': [p['name'] for p in product],
        'reporter': reporter
    }

In [ ]:
import pandas as pd

file_path = r'C:\Users\MGroup\Downloads\VULN tokens.csv'
df = pd.read_csv(file_path)

issues = list(dict.fromkeys(df['Ссылка на youtrack'].values.tolist()))
issues = [ i.replace('https://youtrack.wildberries.ru/issue/', '') for i in issues]
issues

In [ ]:
df

In [ ]:
issues_authority = []
for id in issues:
    issue = youtrack_client.get_issue(id)
    if issue is None:
        print(f'Issue {id} was not found.')
        continue
    issue_authority = get_issue_authority(issue)
    issues_authority.append({
         'issue_id': id,
        **issue_authority
    })


issues_authority

In [ ]:
df['issue_id'] = df['Ссылка на youtrack'].str.extract(r'issue/(VR-\d+)')

json_df = pd.DataFrame(issues_authority)
merged_df = pd.merge(df, json_df, on="issue_id", how="left")

merged_df.to_csv('vulns.csv')

In [ ]:
merged_df['products'].apply(assign_team)

In [ ]:

# Define a function to determine the team based on the product
def assign_team(products):
    if any(product in ['admin', 'vsemrabota', 'wbteam'] for product in products):
        return 'WB Team'
    elif 'ats' in products:
        return 'ATS Team'
    else:
        return 'Empty Team'

# Apply the function to create a new 'Team' column
merged_df['Team'] = merged_df['products'].apply(assign_team)

# Group by 'Team' and then by 'issue_id', and collect tokens
grouped_data = merged_df.groupby(['Team', 'issue_id'])['Секрет'].apply(list)

output_format = {}
for (team, issue_id), tokens in grouped_data.items():
    if team not in output_format:
        output_format[team] = []
    output_format[team].append((issue_id, tokens))

# Print in the desired format
for team, issues in output_format.items():
    print(f"{team}:")
    for issue_id, tokens in issues:
        print(f"{issue_id}")
        for token in tokens:
            print(f"- {token}")
    print("\n")

In [ ]:
merged_df.to_csv('vulns.csv')

In [ ]:
issues = youtrack_client.get('/api/issues', params={'query': 'Business Line:HR   State: Resolved sort by: {Threat Score} desc'})
print(issues)

In [ ]:
youtrack_client.get_issues('Business Line:HR   State: Resolved sort by: {Threat Score} desc')

In [13]:
def get_project_by_name(project_short_name: str):
    projects = youtrack_client.get_project_by_name(project_short_name)

    target_project = next(project for project in projects if project['shortName'] == project_short_name)
    return target_project
   
get_project_by_name('VR')

{'shortName': 'VR', 'name': 'VsemRabota', 'id': '77-167', '$type': 'Project'}

In [16]:
import json

print(.dumps(youtrack_client.get_issue('VR-5418'), indent=2))

{
  "idReadable": "VR-5418",
  "summary": "[BE] - AuthV3 - \u0418\u043d\u0442\u0435\u0433\u0440\u0430\u0446\u0438\u044f auth v3 \u0432 crm-api",
  "reporter": {
    "fullName": "\u0427\u0430\u043f\u043b\u0438\u043d\u0441\u043a\u0438\u0439 \u042f\u0440\u043e\u0441\u043b\u0430\u0432 \u042d\u0434\u0443\u0430\u0440\u0434\u043e\u0432\u0438\u0447",
    "login": "Yaraslau_Chaplinski",
    "id": "24-3942",
    "$type": "User"
  },
  "customFields": [
    {
      "projectCustomField": {
        "field": {
          "name": "Type",
          "id": "42-2",
          "$type": "CustomField"
        },
        "id": "78-1083",
        "$type": "EnumProjectCustomField"
      },
      "value": {
        "localizedName": null,
        "name": "Task",
        "color": {
          "id": "0",
          "$type": "FieldStyle"
        },
        "id": "53-3712",
        "$type": "EnumBundleElement"
      },
      "id": "78-1083",
      "$type": "SingleEnumIssueCustomField"
    },
    {
      "projectCustomFi

In [ ]:
resp = youtrack_client.get_issues('Reviewer [HR]: Valentin_Marchuk, Vansevich.E, Mikhail_Shogin, grigorev.mark , Vitaliy_Guselnikov, aleksandrov.e23, krupenko.ilya, mustafetov.n , goncharov.v38, korolev.artem14  and State: Review ')
resp

In [ ]:
review_items = []

for item in resp:
    print(item['id'])
    issue = youtrack_client.get_issue(item['id'])
    history = youtrack_client.get_issue_history(issue["idReadable"])
    
    latest_review_time, hours_passed = get_latest_review_time_and_passed_hours(history)
    
    reviewer = next(field['value']['login'] for field in issue['customFields'] if field['projectCustomField']['field']['name'] == 'Reviewer [HR]')
    
    review_items.append({
        'reviewer': reviewer,
        'issue_id': issue["idReadable"],
        'summary': issue["summary"],
        'latest_review_time': latest_review_time,
        'hours_passed': hours_passed
    })

In [ ]:
youtrack_client.get_field('VR-8939', '86-578')

In [ ]:
issue = youtrack_client.get_issue('92-1177453')
history = youtrack_client.get_issue_history(issue["idReadable"])

latest_review_time, hours_passed = get_latest_review_time_and_passed_hours(history)

reviewer = next(field['value']['login'] for field in issue['customFields'] if field['projectCustomField']['field']['name'] == 'Reviewer [HR]')

item = {
    'reviewer': reviewer,
    'issue_id': issue["idReadable"],
    'summary': issue["summary"],
    'latest_review_time': latest_review_time,
    'hours_passed': hours_passed
}
# print(f'[{issue["idReadable"]}] {issue["summary"]} {reviewer}')

item

In [ ]:
youtrack_client.get_issue('VR-8939')

In [ ]:
history = youtrack_client.get_issue_history('VR-8816')

history

In [ ]:
activity_log = history

In [ ]:
from datetime import datetime, timedelta

def get_latest_review_time_and_passed_hours(activity_log):
    # There was an error due to the first item in the log having 'added' as an integer. Let's correct this and process again.
    latest_review_timestamp = None
    for activity in activity_log:
        if 'added' in activity and isinstance(activity['added'], list) and any('name' in d and d['name'] == 'Review' for d in activity['added']):
            latest_review_timestamp = activity['timestamp']
    
    # Convert the timestamp from milliseconds to seconds
    latest_review_timestamp_seconds = latest_review_timestamp / 1000 if latest_review_timestamp else None
    
    # Calculate the hours passed from the latest status change to "Review" until now
    if latest_review_timestamp_seconds:
        latest_review_time = datetime.utcfromtimestamp(latest_review_timestamp_seconds)
        current_time = datetime.utcnow()
        hours_passed = (current_time - latest_review_time).total_seconds() / 3600
    else:
        hours_passed = None
    
    return latest_review_time, hours_passed

get_latest_review_time_and_passed_hours(activity_log)

In [ ]:
review_items

In [ ]:
from collections import defaultdict
from datetime import datetime
import re

def escape_markdown(text):
    # Define the pattern for special Markdown characters that need to be escaped
    special_chars_pattern = r'([_*\[\]()~`>#+\-=|{}.!])'
    # Replace each special character with its escaped version
    escaped_text = re.sub(special_chars_pattern, r'\\\1', text)
    return escaped_text

mapped_user_names = {
    'Valentin_Marchuk': 'valuamba',
    'Vansevich.E': 'evansevich',
    'Mikhail_Shogin': 'mshogin'
}

grouped_issues = defaultdict(list)
for issue in review_items:
    grouped_issues[issue['reviewer']].append(issue)

# "[*" + escapeMarkdown(issueStr) + "*]"  + "(" + issueUrl + ")\n\n"

# Sort and format output
output = "\#review\n\nЗадачи на ревью:\n\n"
for reviewer, issues in grouped_issues.items():
    output += f"@{mapped_user_names[reviewer]}:\n"
    # Sort issues by hours_passed
    sorted_issues = sorted(issues, key=lambda x: x['hours_passed'], reverse=True)
    for issue in sorted_issues:
        issue_url = f'https://youtrack.wildberries.ru/issue/{issue["issue_id"]}'
        time_passed = f"{int(issue['hours_passed'] / 24)}d" if issue['hours_passed'] > 24 else f"{round(issue['hours_passed'])}h"
        output += f"{time_passed} [*\[{escape_markdown(issue['issue_id'])}\]*]({issue_url}) {escape_markdown(issue['summary'])}\n"
    output += '\n'

print(output.strip())

In [ ]:
https://t.me/c/2011635320/2/3

In [ ]:
import asyncio
import telegram

test_chat = -1002011635320
test_thread = 2

# https://t.me/c/1593412877/10060/10061
bot = telegram.Bot("6779084548:AAEzWtAFQphlMonph8R-pw9IlU9YzFPal2k")


await bot.send_message(text=output.strip(), chat_id=-1001593412877, message_thread_id=10060, parse_mode='MarkdownV2')

### Custom Fields

In [19]:
class CustomField:
    def __init__(self, name, value, field_type="field_type"):
        self.name = name
        self.value = value
        self.field_type = field_type

class EnumField(CustomField):
    def __init__(self, name, value):
        super().__init__(name, {"name": value}, field_type="SingleEnumIssueCustomField")

class UserField(CustomField):
    def __init__(self, name, login):
        super().__init__(name, {"login": login}, field_type="SingleUserIssueCustomField")

class MultiEnumField(CustomField):
    def __init__(self, name, values):
        super().__init__(name, [{"name": value} for value in values], field_type="MultiEnumIssueCustomField")

class PeriodField(CustomField):
    def __init__(self, name, presentation):
        super().__init__(name, {"presentation": presentation}, field_type="PeriodIssueCustomField")

class DateField(CustomField):
    def __init__(self, name, value=None):
        super().__init__(name, value, field_type="DateIssueCustomField")

class SimpleField(CustomField):
    def __init__(self, name, value):
        super().__init__(name, value, field_type="SimpleIssueCustomField")


In [45]:
from datetime import datetime, timezone
import pytz

class VRIssueCustomFields:
    def __init__(self, assignee, type, devs, reviewer_hr, priority, state, stream, estimation, complexity, product, environments, services, start_date=None, due_date=None):
        self.assignee = assignee
        self.type = type
        self.devs = devs
        self.reviewer_hr = reviewer_hr
        self.priority = priority
        self.state = state
        self.stream = stream
        self.estimation = estimation
        self.complexity = complexity
        self.product = product
        self.environments = environments
        self.services = services
        self.start_date = self.convert_date_to_utc_millis(start_date)
        self.due_date = self.convert_date_to_utc_millis(due_date)

    def serialize(self):
        custom_fields = [
            EnumField("Type", self.type),
            EnumField("Devs", self.devs),
            UserField("Assignee", self.assignee),
            UserField("Reviewer [HR]", self.reviewer_hr),
            EnumField("Priority", self.priority),
            EnumField("State", self.state),
            EnumField("Stream", self.stream),
            PeriodField("Estimation", self.estimation),
            EnumField("Complexity", self.complexity),
            MultiEnumField("Product", [self.product]),
            EnumField("Environments", self.environments),
            MultiEnumField("Services", self.services.split(',')),
            DateField("Start Date", self.start_date),
            DateField("Due Date", self.due_date),
        ]

        # Filtering out None values and adjusting for field type
        serialized_fields = []
        for field in custom_fields:
            if field.value is not None:
                serialized_fields.append({"name": field.name, "value": field.value, "$type": field.field_type})
                # if isinstance(field, DateField):
                #     # Ensure date fields are serialized with proper value format
                #     serialized_fields.append({"name": field.name, "value": {"$type": "Long", "value": field.value}, "$type": field.field_type})
                # else:
                #     serialized_fields.append({"name": field.name, "value": field.value, "$type": field.field_type})
        
        return serialized_fields

    @staticmethod
    def convert_date_to_utc_millis(date_str):
        if date_str is None:
            return None
        # Assuming the input format is 'YYYY-MM-DD'
        dt = datetime.strptime(date_str, '%Y-%m-%d')
        # Adjust for UTC+3
        tz_utc_3 = pytz.timezone('Europe/Moscow')
        dt = tz_utc_3.localize(dt)
        # Convert to UTC
        dt_utc = dt.astimezone(pytz.utc)
        # Convert to milliseconds since epoch
        return int(dt_utc.timestamp() * 1000)

# Adapt EnumField, UserField, PeriodField, MultiEnumField, DateField, and SimpleField definitions as required


### Create Task

In [41]:
# Instantiate VRIssueCustomFields with hardcoded values
issue_custom_fields = VRIssueCustomFields(
    assignee="grigorev.mark",
    type="Task",
    devs="BE",
    reviewer_hr="Markov.Kirill3",
    priority="Show-stopper",
    state="Review",
    stream="Product",
    estimation="4h",
    complexity="Вроде не изян",
    product="all",
    environments="prod",
    services="auth",
    start_date=None,  # Assuming these fields are optional and can be None
    due_date='2024-05-19'
)

# To get the serialized form suitable for creating an issue in YouTrack
serialized_fields = issue_custom_fields.serialize()

# `serialized_fields` now contains the custom fields in the format expected by YouTrack API.


In [40]:
serialized_fields

[{'name': 'Type',
  'value': {'name': 'Task'},
  '$type': 'SingleEnumIssueCustomField'},
 {'name': 'Devs',
  'value': {'name': 'BE'},
  '$type': 'SingleEnumIssueCustomField'},
 {'name': 'Assignee',
  'value': {'login': 'grigorev.mark'},
  '$type': 'SingleUserIssueCustomField'},
 {'name': 'Reviewer [HR]',
  'value': {'login': 'Markov.Kirill3'},
  '$type': 'SingleUserIssueCustomField'},
 {'name': 'Priority',
  'value': {'name': 'Show-stopper'},
  '$type': 'SingleEnumIssueCustomField'},
 {'name': 'State',
  'value': {'name': 'Review'},
  '$type': 'SingleEnumIssueCustomField'},
 {'name': 'Stream',
  'value': {'name': 'Product'},
  '$type': 'SingleEnumIssueCustomField'},
 {'name': 'Estimation',
  'value': {'presentation': '4h'},
  '$type': 'PeriodIssueCustomField'},
 {'name': 'Complexity',
  'value': {'name': 'Вроде не изян'},
  '$type': 'SingleEnumIssueCustomField'},
 {'name': 'Product',
  'value': [{'name': 'all'}],
  '$type': 'MultiEnumIssueCustomField'},
 {'name': 'Environments',
  'val

In [46]:
youtrack_client = YouTrackClient(config=config_model.YouTrack)


In [47]:

# Usage example:
issue_custom_fields = VRIssueCustomFields(
    assignee="grigorev.mark",
    type="Task",
    devs="BE",
    reviewer_hr="Valentin_Marchuk",
    priority="Normal",
    state="Review",
    stream="Product",
    estimation="4h",
    complexity="Вроде не изян",
    product="all",
    environments="prod",
    services="auth",
    start_date=None,
    due_date='2024-05-19',
)

response_data = youtrack_client.create_issue_with_custom_fields(issue_custom_fields, "77-167", "Test 1", "Test 2")
print(response_data)


{'id': '92-1208267', '$type': 'Issue'}


In [48]:
youtrack_client.get_issue('92-1208267')

{'idReadable': 'VR-9078',
 'summary': 'Test 1',
 'reporter': {'fullName': 'Марчук Валентин',
  'login': 'Valentin_Marchuk',
  'id': '24-5060',
  '$type': 'User'},
 'customFields': [{'projectCustomField': {'field': {'name': 'Type',
     'id': '42-2',
     '$type': 'CustomField'},
    'id': '78-1083',
    '$type': 'EnumProjectCustomField'},
   'value': {'localizedName': None,
    'name': 'Task',
    'color': {'id': '0', '$type': 'FieldStyle'},
    'id': '53-3712',
    '$type': 'EnumBundleElement'},
   'id': '78-1083',
   '$type': 'SingleEnumIssueCustomField'},
  {'projectCustomField': {'field': {'name': 'Devs',
     'id': '42-764',
     '$type': 'CustomField'},
    'id': '78-1985',
    '$type': 'EnumProjectCustomField'},
   'value': {'localizedName': None,
    'name': 'BE',
    'color': {'id': '0', '$type': 'FieldStyle'},
    'id': '53-1464',
    '$type': 'EnumBundleElement'},
   'id': '78-1985',
   '$type': 'SingleEnumIssueCustomField'},
  {'value': {'name': 'Григорьев Марк',
    'fullN